# Setup do Projeto

In [43]:
# Testando CUDA
import torch
print(f"CUDA está disponível: {torch.cuda.is_available()}\nDevice count: {torch.cuda.device_count()}\nNome do dispositivo: {torch.cuda.get_device_name(0)}\nVersao do PyTorch: {torch.__version__}")

CUDA está disponível: True
Device count: 2
Nome do dispositivo: AMD Radeon RX 7800 XT
Versao do PyTorch: 2.9.1+rocm7.2.0.git7e1940d4


In [29]:
# Setup Agnostic Code for GPU/CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [38]:
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import v2
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [48]:
# Constantes
DATA_PATH = '/home/jose-vitor/Documents/Cityscapes_Dataset'
BATCH_SIZE = 64

# Importar o Dataset Cityscapes

In [78]:
# transformando dados em tensores e aplicando data augmentation
# O dataset original possui tamanho 1024x2048

# -- DATA TRANSFORMS -- #
train_transform = v2.Compose([
    v2.Resize(size=(256,512)),
    v2.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET), # aplica data augmentation
    v2.PILToTensor(), # converte imagem PIL para tensor
    v2.ToDtype(torch.float32, scale=True) # normaliza para [0,1]
])

val_transform = v2.Compose([
    v2.Resize(size=(256,512)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True)
])

# -- TARGET TRANFROMS -- #
lable_squeeze = lambda x: x.squeeze(dim=0) # remove canal extra desnecessário na segmentação

target_transform = v2.Compose([
    #v2.Resize(size=(256,512)),
    v2.PILToTensor(), # converte segmentação PIL para tensor
    v2.Lambda(lable_squeeze) # remove canal extra desnecessário na segmentação
])

In [79]:
train_dataset = datasets.Cityscapes(DATA_PATH,
                                    split = 'train',
                                    mode='fine',
                                    target_type='semantic',
                                    transform=train_transform,
                                    target_transform=target_transform)
val_dataset = datasets.Cityscapes(DATA_PATH,
                                    split = 'val',
                                    mode='fine',
                                    target_type='semantic',
                                    transform=val_transform,
                                    target_transform=target_transform)
test_dataset = datasets.Cityscapes(DATA_PATH,
                                    split = 'test',
                                    mode='fine',
                                    target_type='semantic',
                                    transform=val_transform,
                                    target_transform=target_transform)

print(f"Train Dataset: {train_dataset}\nVal Dataset: {val_dataset}\nTest Dataset: {test_dataset}")

Train Dataset: Dataset Cityscapes
    Number of datapoints: 2975
    Root location: /home/jose-vitor/Documents/Cityscapes_Dataset
    Split: train
    Mode: gtFine
    Type: ['semantic']
    StandardTransform
Transform: Compose(
                 Resize(size=[256, 512], interpolation=InterpolationMode.BILINEAR, antialias=True)
                 AutoAugment(interpolation=InterpolationMode.NEAREST, policy=AutoAugmentPolicy.IMAGENET)
                 PILToTensor()
                 ToDtype(scale=True)
           )
Target transform: Compose(
                        PILToTensor()
                        Lambda(<lambda>, types=['object'])
                  )
Val Dataset: Dataset Cityscapes
    Number of datapoints: 500
    Root location: /home/jose-vitor/Documents/Cityscapes_Dataset
    Split: val
    Mode: gtFine
    Type: ['semantic']
    StandardTransform
Transform: Compose(
                 Resize(size=[256, 512], interpolation=InterpolationMode.BILINEAR, antialias=True)
                 PI

In [ ]:
# Imprimindo imagens e segmentações do dataset
def img_show(imgs: list[torch.Tensor], smnts: list[torch.Tensor] = None, n: int=5):
    plt.figure(figsize=(15,10))
    for i in range(n):
        plt.subplot(n, 2, 2*i+1)
        plt.imshow(imgs[i].permute[1,2,0]) # permute para mudar a ordem dos canais e converter um tensor para imagem
        

ValueError: too many values to unpack (expected 2)

In [ ]:
img, smnt = train_dataset[0]
smnt.shape



torch.Size([1024, 2048])

Criar dataloaders

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=)